In [9]:
from pathlib import *
import sqlalchemy as sa
import pandas as pd

database_management_sys = sa.create_engine("sqlite:///../database-code/data/se/main.db")
connect_dbms_to_db = database_management_sys.connect()

In [13]:
trial_tournament_set = """
    SELECT * FROM folk_theorem_experiment
    WHERE tournament_player_set = '156'
    AND noise = '0.0'
    AND player_strategy_name = 'Defector'
"""
trial_data = pd.read_sql(trial_tournament_set, connect_dbms_to_db)
trial_data

,experiment_number,number_of_players,tournament_player_set,player_strategy_name,is_long_run_time,is_stochastic,memory_depth_of_strategy,prob_of_game_ending,payoff_matrix,num_of_repetitions,num_of_equilibria,nash_equilibria,least_prob_of_defection,greatest_prob_of_defection,noise,warning_message
0,171600,8,156,Defector,0,0,0,0.001000,2.98713198 2.4970386 0.00926656 2.9868275...,500,6,0.00000000e+00 0.00000000e+00 1.00000000...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (6) equil...
1,171601,8,156,Defector,0,0,0,0.011081,2.90458327 2.52269817 0.04820977 2.9008351...,500,4,0. 0. 1. 0. ...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (4) equil...
2,171602,8,156,Defector,0,0,0,0.021162,2.81805332 2.53305455 0.0910335 2.8001180...,500,6,0.00000000e+00 0.00000000e+00 1.00000000...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (6) equil...
3,171603,8,156,Defector,0,0,0,0.031242,2.78032603 2.54974768 0.10876013 2.7839448...,500,4,0.00000000e+00 0.00000000e+00 1.00000000...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (4) equil...
4,171604,8,156,Defector,0,0,0,0.041323,2.67889355 2.61037339 0.13824453 2.7450524...,500,4,0.00000000e+00 0.00000000e+00 1.00000000...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (4) equil...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,171695,8,156,Defector,0,0,0,0.958677,1.03266667e+00 4.89200000e+00 9.77000000...,500,5,0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0...,0.0,1.0,0.0,None
96,171696,8,156,Defector,0,0,0,0.968758,1.04466667 4.944 0.98366667 1.036 ...,500,4,0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0...,0.0,1.0,0.0,[RuntimeWarning('\nAn even number of (4) equil...
97,171697,8,156,Defector,0,0,0,0.978838,1.01 4.96 0.98133333 1.014 ...,500,9,0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0...,0.0,1.0,0.0,None
98,171698,8,156,Defector,0,0,0,0.988919,1.01000000e+00 4.94000000e+00 9.95000000...,500,9,0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0...,0.0,1.0,0.0,None


In [4]:
def find_min_p_thresh(tournament_set):
    """
    A function to find the minimum p-threshold from the database, where:
    
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level.
    """
    defection_prob_at_smallest_prob_game_ending = tournament_set["least_prob_of_defection"][0]
    any_equal_prob = tournament_set[tournament_set["least_prob_of_defection"] == defection_prob_at_smallest_prob_game_ending]
    if len(tournament_set["least_prob_of_defection"]) == len(any_equal_prob):
        if defection_prob_at_smallest_prob_game_ending == 0:
        # that is, if the least probability of defection is constant at 0 for all game ending probabilities.    
            min_p_thresh = None
        else:
        # the least probability of defection is equal to some non-zero constant for all game ending probabilities.
            min_p_thresh = min(tournament_set["prob_of_game_ending"])
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        min_p_thresh = min(prob_of_defection_is_non_zero["prob_of_game_ending"])
    return(min_p_thresh)

In [15]:
def find_max_p_thresh(tournament_set):
    """
    A function to find the maximum p-threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level.
    """
    defection_prob_at_smallest_prob_game_ending = tournament_set["least_prob_of_defection"][0]
    any_equal_prob = tournament_set[tournament_set["least_prob_of_defection"] == defection_prob_at_smallest_prob_game_ending]
    if len(tournament_set["least_prob_of_defection"]) == len(any_equal_prob):
        if defection_prob_at_smallest_prob_game_ending == 0:
        # that is, if the least probability of defection is constant at 0 for all game ending probabilities.    
            max_p_thresh = None
        else:
        # the least probability of defection is equal to some non-zero constant for all game ending probabilities.
            max_p_thresh = max(tournament_set["prob_of_game_ending"])
    else:
        prob_of_defection_is_zero = tournament_set[tournament_set["least_prob_of_defection"] == 0]
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        max_zero_defection_game_ending = max(prob_of_defection_is_zero["prob_of_game_ending"])
        if len(prob_of_defection_is_non_zero[prob_of_defection_is_non_zero["prob_of_game_ending"] > max_zero_defection_game_ending]["prob_of_game_ending"]) == 0:
            max_p_thresh = max(prob_of_defection_is_non_zero["prob_of_game_ending"])
        else:    
            max_p_thresh = min(prob_of_defection_is_non_zero[prob_of_defection_is_non_zero["prob_of_game_ending"] > max_zero_defection_game_ending]["prob_of_game_ending"])
    return(max_p_thresh)

In [6]:
def find_mean_p_thresh(tournament_set, min_p_thresh, max_p_thresh):
    """
    A function to find the mean p-threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level;
        "min_p_thresh" is a numeric variable between 0 and 1 indicating the minimum p-threshold; and
        "max_p_thresh" is a numeric variable between 0 and 1 indicating the maximum p-threshold.
    """
    if min_p_thresh == max_p_thresh:
        mean_p_thresh = min_p_thresh
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        non_zero_defection_prob_between_min_max = prob_of_defection_is_non_zero[(prob_of_defection_is_non_zero["prob_of_game_ending"] >= min_p_thresh) & (prob_of_defection_is_non_zero["prob_of_game_ending"] <= max_p_thresh)]
        mean_p_thresh = non_zero_defection_prob_between_min_max["prob_of_game_ending"].mean()
    return(mean_p_thresh)

In [7]:
def find_median_p_thresh(tournament_set, min_p_thresh, max_p_thresh):
    """
    A function to find the median p_threshold from the database, where:
        "tournament_set" is a pandas dataframe containing all the information from the database for one particular set of strategies at one particular noise level;
        "min_p_thresh" is a numeric variable between 0 and 1 indicating the minimum p-threshold; and
        "max_p_thresh" is a numeric variable between 0 and 1 indicating the maximum p-threshold.
    """
    if min_p_thresh == max_p_thresh:
        median_p_thresh = min_p_thresh
    else:
        prob_of_defection_is_non_zero = tournament_set[tournament_set["least_prob_of_defection"] != 0]
        non_zero_defection_prob_between_min_max = prob_of_defection_is_non_zero[(prob_of_defection_is_non_zero["prob_of_game_ending"] >= min_p_thresh) & (prob_of_defection_is_non_zero["prob_of_game_ending"] <= max_p_thresh)]
        median_p_thresh = non_zero_defection_prob_between_min_max["prob_of_game_ending"].median()
    return(median_p_thresh)

In [16]:
min_p_thresh = find_min_p_thresh(tournament_set=trial_data)
max_p_thresh = find_max_p_thresh(tournament_set=trial_data)
mean_p_thresh = find_mean_p_thresh(tournament_set=trial_data, min_p_thresh=min_p_thresh, max_p_thresh=max_p_thresh)
median_p_thresh = find_median_p_thresh(tournament_set=trial_data, min_p_thresh=min_p_thresh, max_p_thresh=max_p_thresh)

min_p_thresh, max_p_thresh, mean_p_thresh, median_p_thresh

(0.3235858585858586,
 0.3639090909090909,
 0.34374747474747475,
 0.34374747474747475)